In [ ]:
import pandas as pd
import datetime
import numpy as np
# Testing

# Linkedin Data from Kaggle

In [ ]:
data = pd.read_csv("job postings 2023 24/postings.csv")

In [ ]:
data.columns

In [ ]:
def convert_unix_to_ddmmyyyy(unix_time):
    # Convert from milliseconds to seconds
    unix_time_seconds = unix_time / 1000.0
    
    # Convert to datetime and handle errors
    formatted_dates = pd.to_datetime(unix_time_seconds, unit='s', errors='coerce')
    
    # Format as 'dd-mm-yyyy'
    return formatted_dates.dt.strftime('%d-%m-%Y')

In [ ]:
raw_data = data.copy()

In [ ]:
raw_data['original_listed_time_mod'] = convert_unix_to_ddmmyyyy(raw_data['original_listed_time'])

In [ ]:
raw_data['original_listed_time_mod'] =pd.to_datetime(raw_data['original_listed_time_mod'], format='%d-%m-%Y', errors='coerce')

In [ ]:
raw_data['original_listed_time_mod'].min()

In [ ]:
raw_data['original_listed_time_mod'].max()

# Data Scraped from ScrapingDog API

In [ ]:
import requests

def fetch_all_linkedin_jobs(api_key, fields, geoids, pages):
    url = "https://api.scrapingdog.com/linkedinjobs/"
    all_data = []  # Store data from all requests

    # Iterate over each combination of field, geoid, and page
    for field in fields:
        for geoid in geoids:
            for page in pages:
                params = {
                    "api_key": api_key,
                    "field": field,
                    "geoid": geoid,
                    "page": page
                }
                
                response = requests.get(url, params=params)
                
                if response.status_code == 200:
                    # Append data from each request to the list
                    all_data.extend(response.json())
                else:
                    print(f"Request failed for field: {field}, geoid: {geoid}, page: {page} with status code: {response.status_code}")

    return all_data

# Usage example
api_key = "671da7467fddaf7ca053001a"
fields = ["java", "data science", "machine learning"]
geoids = ["100293800","103736294","102277331"]  # NV, Colorado, California
pages = ["1", "2", "3"]

# data = fetch_all_linkedin_jobs(api_key, fields, geoids, pages)



In [ ]:
if data:
    new_Data = pd.DataFrame(data)
    new_Data.to_csv("261024_Job_Data.csv")

# Data Preprocessing

## Filtering Data

Filtering Function Overview
The filter_data function dynamically filters a DataFrame based on a specified list of columns and corresponding conditions. This flexibility allows for customized data extraction based on various criteria, such as dates, maximum or minimum values, and frequent items.

Function Parameters
df (pd.DataFrame): The input DataFrame containing the data you want to filter.

columns (list): A list of column names to apply filters on. Each column corresponds to a condition in the conditions list.

conditions (list): A list of conditions, where each element is a tuple specifying the operation and the value for filtering. Each tuple matches one of the columns in the columns list.


In [ ]:
import pandas as pd

def filter_data(df, columns, conditions):
    """
    Filters a DataFrame based on a list of columns and their conditions.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to filter.
    - columns (list): List of column names to apply filters on.
    - conditions (list): List of conditions where each element is a tuple in the format:
                         (condition, value), for example ('>', '2024-04-20').
                         Each tuple should correspond to a column in the columns list.
    
    Returns:
    - pd.DataFrame: A filtered DataFrame.
    """
    # Loop over each column and its corresponding condition
    for column, (condition, value) in zip(columns, conditions):
        if condition == '>':
            df = df[df[column] > value]
        elif condition == '<':
            df = df[df[column] < value]
        elif condition == '==':
            df = df[df[column] == value]
        elif condition == '>=':
            df = df[df[column] >= value]
        elif condition == '<=':
            df = df[df[column] <= value]
        elif condition == '!=':
            df = df[df[column] != value]
        elif operation == 'max':
            max_value = df[column].max()
            df = df[df[column] == max_value]
        elif operation == 'min':
            min_value = df[column].min()
            df = df[df[column] == min_value]
        elif operation == 'top_n':
            top_n_counts = df[column].value_counts().nlargest(value)
            df = df[df[column].isin(top_n_counts.index)]
        else:
            print(f"Invalid condition '{condition}' for column '{column}'")
    
    return df


## Filtering Data by Date

In [ ]:
# Load the data
scraped_data = pd.read_csv("C:/Users/DELL/Downloads/261024_Job_Data.csv")

# Convert date column to datetime if filtering by date
scraped_data['job_posting_date'] = pd.to_datetime(scraped_data['job_posting_date'], errors='coerce')

# Specify columns and conditions
columns = ['job_posting_date']
conditions = [('>', '2024-04-20')]

# Apply the filter function
filtered_scraped_data = filter_data(scraped_data, columns, conditions)

# Save the filtered data
filtered_scraped_data.to_csv("filtered_scraped_job_data.csv", index=False)

# Print the filtered data
print(filtered_scraped_data)